In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [32]:
from dataclasses import dataclass
import httpx
import pandas as pd

In [17]:
@dataclass
class Datasource:
    name: str = ""
    meta: str = ""
    api_url: str = ""
    api_key: str = ""
    api_id: str = ""
    header: str = ""


In [13]:
data = Datasource(name="NSW Car Park Occupancy", meta="https://opendata.transport.nsw.gov.au/dataset/car-park-api")

In [14]:
data.api_url = "https://api.transport.nsw.gov.au/v1/carpark"

In [15]:
data.api_id = "databooth-nsw-open-data"

In [16]:
data.api_key = "fwyH5kqV8mkdDwOfJlwoo11ubCivNjqmWeUk"    # TODO: Move to secret

In [21]:
headers = {'Accept': 'application/json', 'Authorization': f'apikey {data.api_key}'}


In [22]:
headers

{'Accept': 'application/json',
 'Authorization': 'apikey fwyH5kqV8mkdDwOfJlwoo11ubCivNjqmWeUk'}

In [63]:
def get_data_for_api(url, headers):
    with httpx.Client() as client:
        r = client.get(url, headers=headers)
        if r.status_code == httpx.codes.OK:
            return pd.DataFrame.from_dict(data=r.json(), orient="index", columns=["name"])

In [73]:
carpark_names_df = get_data_for_api(data.api_url, headers)

In [74]:
carpark_names_df["id"] = carpark_names_df.index
carpark_names_df.reset_index(inplace=True)
carpark_names_df.drop("index", axis=1)


,name,id
0,Tallawong Station Car Park,1
1,Warriewood Car Park,10
2,Narrabeen Car Park,11
3,Mona Vale Car Park,12
4,Dee Why Car Park,13
5,West Ryde Car Park,14
6,Sutherland East Parade Car Park,15
7,Leppington Car Park,16
8,Edmondson Park South Car Park,17
9,St Marys Car Park,18
